In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime

import sys
sys.path.append('../../')

from models.seir import SIRD, SEIR_Testing

from main.seir.fitting import get_variable_param_ranges, single_fitting_cycle

from data.dataloader import Covid19IndiaLoader
from data.processing import get_dataframes_cached

from utils.loss import Loss_Calculator

In [ ]:
loader = Covid19IndiaLoader()
# dataframes = loader.get_covid19india_api_data()
dataframes = get_dataframes_cached()

In [ ]:
predictions_dict = {}

In [ ]:
districts_to_show = [('Maharashtra', 'Pune')]

In [ ]:
filename = '../../data/data/mumbai/case_summary_expanded_2006.csv'
filename = None
data_from_tracker = False

In [ ]:
lc = Loss_Calculator()

### Recovered and deceased

#### SIRD

In [ ]:
variable_param_ranges = {
    'lockdown_R0': (1, 6),
    'T_inc': (4, 16),
    'T_inf': (10, 60),
    'T_fatal': (260, 300)
}
variable_param_ranges = get_variable_param_ranges(variable_param_ranges)
which_compartments=['deceased', 'recovered']

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(
        dataframes, state, district, data_from_tracker=data_from_tracker, granular_data=False, filename=filename, #Data
        model=SIRD, variable_param_ranges=variable_param_ranges, #Choose Model and Ranges
        train_period=7, val_period=7, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
        which_compartments=which_compartments, #Compartments to Apply Loss on 
        smooth_jump=False, smoothing_length=33, smoothing_method='weighted') #Smoothing
    
    predictions_dict[(state, district)]['state'] = state
    predictions_dict[(state, district)]['dist'] = district
    predictions_dict[(state, district)]['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict[(state, district)]['datasource'] = 'covid19api' if predictions_dict[(state, district)]['m1']['data_from_tracker'] else 'municipality'
    predictions_dict[(state, district)]['variable_param_ranges'] = predictions_dict[(state, district)]['m1']['variable_param_ranges']

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')

#### SEIR_Testing

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(
        dataframes, state, district, data_from_tracker=data_from_tracker, granular_data=False, filename=filename, #Data
        model=SEIR_Testing, variable_param_ranges=None, #Choose Model and Ranges
        train_period=7, val_period=7, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
        which_compartments=which_compartments, #Compartments to Apply Loss on 
        smooth_jump=False, smoothing_length=33, smoothing_method='weighted') #Smoothing
    
    predictions_dict[(state, district)]['state'] = state
    predictions_dict[(state, district)]['dist'] = district
    predictions_dict[(state, district)]['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict[(state, district)]['datasource'] = 'covid19api' if predictions_dict[(state, district)]['m1']['data_from_tracker'] else 'municipality'
    predictions_dict[(state, district)]['variable_param_ranges'] = predictions_dict[(state, district)]['m1']['variable_param_ranges']

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')

### All compartments

#### SIRD

In [ ]:
variable_param_ranges = {
    'lockdown_R0': (1, 2),
    'T_inc': (4, 10),
    'T_inf': (10, 20),
    'T_fatal': (260, 290),
}
variable_param_ranges = get_variable_param_ranges(variable_param_ranges)
which_compartments=['hospitalised', 'total_infected', 'deceased', 'recovered']

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(
        dataframes, state, district, data_from_tracker=data_from_tracker, granular_data=False, filename=filename, #Data
        model=SIRD, variable_param_ranges=variable_param_ranges, #Choose Model and Ranges
        train_period=7, val_period=7, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
        which_compartments=which_compartments, #Compartments to Apply Loss on 
        smooth_jump=False, smoothing_length=33, smoothing_method='weighted') #Smoothing
    
    predictions_dict[(state, district)]['state'] = state
    predictions_dict[(state, district)]['dist'] = district
    predictions_dict[(state, district)]['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict[(state, district)]['datasource'] = 'covid19api' if predictions_dict[(state, district)]['m1']['data_from_tracker'] else 'municipality'
    predictions_dict[(state, district)]['variable_param_ranges'] = predictions_dict[(state, district)]['m1']['variable_param_ranges']

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')

#### SEIR_Testing

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(
        dataframes, state, district, data_from_tracker=data_from_tracker, granular_data=False, filename=filename, #Data
        model=SEIR_Testing, variable_param_ranges=None, #Choose Model and Ranges
        train_period=7, val_period=7, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
        which_compartments=which_compartments, #Compartments to Apply Loss on 
        smooth_jump=False, smoothing_length=33, smoothing_method='weighted') #Smoothing
    
    predictions_dict[(state, district)]['state'] = state
    predictions_dict[(state, district)]['dist'] = district
    predictions_dict[(state, district)]['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict[(state, district)]['datasource'] = 'covid19api' if predictions_dict[(state, district)]['m1']['data_from_tracker'] else 'municipality'
    predictions_dict[(state, district)]['variable_param_ranges'] = predictions_dict[(state, district)]['m1']['variable_param_ranges']

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')

### Hospitalised and total infected

In [ ]:
variable_param_ranges = {
    'lockdown_R0': (1, 1.5),
    'T_inc': (4, 10),
    'T_inf': (10, 20),
    'T_fatal': (260, 290)
}
variable_param_ranges = get_variable_param_ranges(variable_param_ranges)
which_compartments=['hospitalised', 'total_infected']

In [ ]:
for state, district in districts_to_show:
    predictions_dict[(state, district)] = {}
    predictions_dict[(state, district)]['m1'] = single_fitting_cycle(
        dataframes, state, district, data_from_tracker=data_from_tracker, granular_data=False, filename=filename, #Data
        model=SIRD, variable_param_ranges=variable_param_ranges, #Choose Model and Ranges
        train_period=7, val_period=7, num_evals=1000, initialisation='intermediate', #Optimisation related parameters
        which_compartments=which_compartments, #Compartments to Apply Loss on 
        smooth_jump=False, smoothing_length=33, smoothing_method='weighted') #Smoothing
    
    predictions_dict[(state, district)]['state'] = state
    predictions_dict[(state, district)]['dist'] = district
    predictions_dict[(state, district)]['fitting_date'] = datetime.datetime.now().strftime("%Y-%m-%d")
    predictions_dict[(state, district)]['datasource'] = 'covid19api' if predictions_dict[(state, district)]['m1']['data_from_tracker'] else 'municipality'
    predictions_dict[(state, district)]['variable_param_ranges'] = predictions_dict[(state, district)]['m1']['variable_param_ranges']

In [ ]:
lc.create_loss_dataframe_master(predictions_dict, 'm1')